In [1]:
from jflegDataset import JflegDataset
import pandas as pd
from torch.utils.data import DataLoader
from transformers import BertTokenizer

In [2]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
TRAIN_PATH = "src/dataset/train.csv"
VAL_PATH = "src/dataset/eval.csv"
ds = JflegDataset(TRAIN_PATH, tokenizer)
dl = DataLoader(ds)


In [3]:
import torch
import numpy as np

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
csv = pd.read_csv(TRAIN_PATH)
csv = csv.groupby('input')['target'].agg(np.array).reset_index()
encoded = tokenizer(csv.iloc[4]["input"], return_tensors="pt")
#tokenizer.decode(encoded["input_ids"])
encoded

{'input_ids': tensor([[  101, 12616,   131,   138,  9131,  1107,  1251,  1768,  1209,  1138,
          1618,  2261,  6305,   117,  1104, 16461,  1119,  1209,  1136,  1138,
          1112,  1242,  6305,  1133,  3451,  1122,  1129,   117,  1209,  1129,
          1677,  1618,   119,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [7]:
i = iter(dl)
next(i)
input, targ = next(i)
ds.decode(input["input_ids"][0])

'[CLS] 1 7 - 5 0 people survey to requre all class and selective class. [SEP]'

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

ds_train = JflegDataset(TRAIN_PATH, tokenizer)
ds_eval = JflegDataset(VAL_PATH, tokenizer)

dl_eval = DataLoader(ds_train)
dl_eval = DataLoader(ds_eval)

In [ ]:
from torch import nn, Tensor
import math

# https://pytorch.org/tutorials/beginner/transformer_tutorial.html
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)